In [1]:
import os
import sys
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
import pickle
import astropy.units as u
from astroquery.ipac.irsa import Irsa
ROOT = os.path.join("./")
sys.path.append(ROOT + "lib")
from helpers import get_coordinates
from lightsource import LightSource
from plotly_style import update_layout

buckets = pickle.load(open(ROOT + "cached_data/raw_classes.pkl", "rb"))

In [2]:
def get_trace(lightsource, key):

  x_data = lightsource["norm"]["day"]
  y_data = lightsource["norm"][key]

  sort_idxs = np.argsort(x_data)

  x_data = x_data[sort_idxs]
  y_data = y_data[sort_idxs]


  return go.Scatter(
    x=x_data,
    y=y_data,
    mode='markers',
    marker=dict(size=5, opacity=.75),
    name=key,
  )

def plot_from_lightsource(lightsource, ys=["w1"]):

  fig = go.Figure()

  for y in ys:
    fig.add_trace(get_trace(lightsource, y))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width

  return fig

def plot_from_tensor(data):
  fig = go.Figure()

  w1 = data[:, 0].numpy()
  std = data[:, 1].numpy()


  day = data[:, -1].numpy()


  fig.add_trace(go.Scatter(x=day, y=w1, marker=dict(size=5, opacity=0.7), name="w1mpro z-scored", mode='markers'))
  fig.add_trace(go.Scatter(x=day, y=std, marker=dict(size=5, opacity=0.7), name="w2mpro z-scored", mode='markers'))

  update_layout(fig, legend_out=True)

  fig.layout.width = 800
  fig.layout.height = 0.65 * fig.layout.width


  return fig

In [3]:
# ----------------------- Begin Work on Dataset ----------------------- # 
ex = buckets["nova"]["V1391 Cas"]
flux = ex["raw"]["w1flux"]

ADJ = 0
DIV = 0.001
norm = np.arcsinh((flux - ADJ) /DIV)
tr1 = go.Scatter(
    x=ex["norm"]["day"],
    y=norm,
    marker=dict(size=5, opacity=0.7), name="normalized flux", mode='markers'
)

tr2 = go.Scatter(
    x=ex["norm"]["day"],
    y=flux,
    marker=dict(size=5, opacity=0.7), name="raw flux", mode='markers'
)

fig = plot_from_lightsource(ex, ["w1"])
fig.add_trace(tr1)
fig.add_trace(tr2)
fig.show()

print([n for n in flux])


[0.36001183524309366, 0.4667848781013259, 0.49649717099165586, 0.41948533069913985, 0.39986811763300595, 0.4167895084675743, 0.3947451276975449, 0.5591351284244347, 0.497412595457606, 0.5769215720600425, 0.6878864871647523, 0.672228279049937, 0.7052072658650503, 0.504797015131243, 0.5295619567814962, 0.600228728794441, 0.45031537815011086, 0.4210336220767442, 0.5871063508516983, 0.5290744367402352, 0.7203055785286608, 0.47853838354895123, 2.0659355944235283, 32.47257871751208, 70.06752517848155, 30.473109922648643, 21.434668844642886, 17.136289016573414, 24.74667064201387, 20.09633113977502, 34.69898556554249, 10.146512115023263, 4.75901176297207, 4.320328581785528, 0.7985769640219369, 2.9889371174255124, 5.555418468497403, 3.580278233952341, 3.9184722747839773, 6.803255450218148, 2.964263307710677, 3.6502009875293044, 3.2086088589965867, 4.987888978067695, 2.529978195569321, 4.031965229807509, 4.249293982162744, 2.0735608199879163, 0.21933946420581674, 0.21712847307291797, 0.222390780

In [4]:
c = get_coordinates("19 52 08.25	+27 42 20.9")

tbl = Irsa.query_region(c, catalog="neowiser_p1bs_psd", spatial="Cone",
                            radius=5 * u.arcsec)
tbl = tbl.to_pandas()
tbl = tbl.loc[tbl["qual_frame"]>= 4]
ex = LightSource(tbl, "")
tbl = ex.get_numpy()
labels = DBSCAN(eps=0.9/3600, min_samples=10).fit(tbl[:, :2])
labels = labels.labels_
biggest = np.argmax(np.bincount(labels))
filter_mask = [x == biggest for x in labels]
tbl = ex.get_pandas()[filter_mask]
ex = LightSource(tbl, "")
plot_from_lightsource(ex)


In [8]:
to_flux_w1 = lambda m: 309.54 * 10**(-m / 2.5)

tr = go.Scatter(
    x=tbl["ra"],
    y=tbl["dec"],
    marker=dict(size=np.log(10**6*to_flux_w1(tbl["w1mpro"])) / 1.4, opacity=0.4, color="red"), name="pos", mode='markers',
)

fig = go.Figure()
fig.add_trace(tr)
fig.show()

In [15]:
ex = buckets["transit"]["RS Cha"]
df = ex.get_pandas()
# tr = go.Scatter(x=df["ra"], y=df["dec"], mode="markers")
# go.Figure().add_trace(tr).show()

cluster_tbl = np_tbl[:, :2]
clstr = DBSCAN(eps=0.8/3600, min_samples=16).fit(cluster_tbl) # high minsamples cutting out some very sparse examples

labels = clstr.labels_
filters = [(l, [x == l for x in labels]) for l in set(labels)]
clusters = [(l, df[f]) for l,f in filters]
traces = [
    go.Scatter(x = tbl["ra"], y = tbl["dec"], mode='markers', name=f"Cluster {i}", marker=dict(opacity=0.9))
    for i, tbl in (clusters)
]
fig = go.Figure()
fig.add_traces(traces)
update_layout(fig)
fig.show()